In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import multiprocessing as mp

import pickle
import json
import functools
import matplotlib
import itertools
import copy

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests
import time

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

# Currently, memory growth needs to be the same across GPUs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

1 Physical GPUs, 1 Logical GPUs


In [3]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        
        print(self.interval)

In [4]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

def try_request(req):
    while True:
        response = esi_client.request(req)
        
        if response.status == 200:
            return response.data
        
        print(response.status)
        print(response.header)
        time.sleep(5)

Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [5]:
def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);
    
with open('universe/market_types.txt') as f:
    types = intd(f)

print('Loading constellation data ...')
constellation_ids = esi_client.request(
    app.op['get_universe_constellations']()
).data

constellations = {
    i : try_request(app.op['get_universe_constellations_constellation_id'](constellation_id = i))
    for i in tqdm(constellation_ids)
}

Loading constellation data ...


In [6]:
systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

root = 'Jita'
root_id = [v for v in systems if systems[v]["name"] == root][0]

# Limit to high-sec systems
subselect = [k for k in systems if systems[k]["security_status"] >= 0.5]
subgraph = systems_graph.subgraph(subselect)

# Limit to component connected to Jita
subselect = nx.node_connected_component(subgraph, root_id)
subgraph = systems_graph.subgraph(subselect)
print(len(subgraph))

system_ids = list(subgraph)
type_ids = list(types)

landmark_names = ['Jita', 'Amarr', 'Rens', 'Hek', 'Dodixie', 'Oursulaert', 'Tash-Murkon Prime', 'Agil'];
landmarks = [v["system_id"] for v in systems.values() if v["name"] in landmark_names]

# Compute pair-wise distances
print('Computing system distances')
distances = dict(tqdm(nx.shortest_path_length(subgraph), total = len(subgraph)))

@functools.lru_cache(maxsize=None)
def system_distance(s1, s2):
    #return nx.shortest_path_length(subgraph, s1, s2)
    return distances[s1][s2]

@functools.lru_cache(maxsize=None)
def get_region_id(system_id):
    return constellations[systems[system_id]["constellation_id"]].region_id

@functools.lru_cache(maxsize=None)
def get_station(station_id):
    esi_op = app.op['get_universe_stations_station_id'](station_id = station_id);
    return esi_client.request(esi_op).data

@functools.lru_cache(maxsize=None)
def in_highsec(station_id):
    try:
        return get_station(station_id).system_id in subgraph
    except:
        return False

972
Computing system distances


In [7]:
# Load orders from EVE
region_id = get_region_id(root_id)

print('Loading regions')
#region_ids = esi_client.request(app.op['get_universe_regions']()).data
region_ids = list(set(
    get_region_id(system_id) for system_id in subgraph
))

print('Loading market orders')

def requests_in_region(region_id):
    region = esi_client.request(
        app.op['get_universe_regions_region_id'](region_id = region_id)
    ).data
    
    print('Loading orders in {}'.format(region.name))
    n_pages = esi_client.request(
        app.op['get_markets_region_id_orders'](region_id = region_id)
    ).header['X-pages'][0]

    p = mp.pool.ThreadPool(1)

    requests = [
        app.op['get_markets_region_id_orders'](region_id = region_id, page = page)
        for page in range(1, n_pages + 1)
    ]
    
    return requests

orders = {
    region_id : {
        type_id : list(orders)
        for (type_id, orders) in itertools.groupby(
            (
                result
                for request in tqdm(requests_in_region(region_id), desc = 'Pages')
                for result in try_request(request)
            )
            , lambda x : x.type_id
        )
    }
    for region_id in tqdm(region_ids, desc = 'Regions')
}

Loading regions
Loading market orders


Loading orders in Sinq Laison



Loading orders in The Citadel



Loading orders in The Forge



Loading orders in Genesis


...


Loading orders in Khanid



Loading orders in Tash-Murkon



Loading orders in Kador



Loading orders in Molden Heath



Loading orders in Heimatar


In [113]:
## Structure of the state space
class StateProp:
    def __init__(self, default, convert = (lambda x: x, lambda x: x)):
        self.convert_in = convert[0]
        self.convert_out = convert[1]
        self.default = default

def to_fixed(x):
    if x is None:
        return None
    return State(x)

def to_mutable(x):
    if x is None:
        return none
    return MutableState(x)

# Dict props should be stored as (key, value) tuples
dict_prop = (lambda x: tuple((i for i in x.items())), lambda x: {k:v for (k,v) in x})

state_props = {
    # Position of the ship
    'system' : StateProp(0),
    'station' : StateProp(None),
    
    # Configuration of the ship
    'volume_limit' : StateProp(0),
    'collateral_limit' : StateProp(0),
    
    # Resource balance
    'time_left' : StateProp(0),
    'wallet' : StateProp(0),
    'cargo' : StateProp({}, dict_prop),
    
    # Market cursors
    'market_group' : StateProp(None),
    'market_item' : StateProp(None),
    'last_modified_item' : StateProp(None),
    'updated_orders' : StateProp({}, dict_prop)
}
    
# Implementation of the state space
class State:
    def __init__(self, inp):
        for name, prop in state_props.items():
            inval = getattr(inp, name, prop.default)
            inval = prop.convert_in(inval)
            object.__setattr__(self, '_prop_' + name, inval)
            
        self._encode_cache = None
    
    def __getattr__(self, name):
        if name in state_props:
            return getattr(self, '_prop_' + name)
        else:
            raise AttributeError('Unknown attribute ' + name)
    
    def __setattr__(self, name, val):
        if name in state_props:
            raise AttributeError('Attribute {} is immutable'.format(name))
        
        object.__setattr__(self, name, val)
    
    def __hash__(self):
        h = 0
        
        for name in state_props:
            h = h ^ hash(getattr(self, name))
        
        return h
    
    def __eq__(self, other):
        for name in state_props:
            if getattr(self, name) != getattr(other, name):
                return False
        
        return True
    
    def __repr__(self):
        (vleft, cleft) = self.limits_left()
        vmax = self.volume_limit
        cmax = self.collateral_limit
        v = vmax - vleft
        c = cmax - cleft
        
        def cinfo(cid):
            def locinfo(lid):
                if(self.station == lid):
                    return "Here"
                
                system = get_station(lid).system_id
                if(system == self.system):
                    return "In system"
                
                return "{} jumps".format(nx.shortest_path_length(subgraph, system, self.system))
            
            c = contracts[cid]
                
            return "{cid} ({l1} -> {l2})".format(
                l1 = locinfo(c.start_location_id),
                l2 = locinfo(c.end_location_id),
                cid = cid
            )
        return """
State:
    Time left: {t}
    
    System:  {system}
    Station: {station}
    
    Collateral: {c} / {cmax} ({cleft} left)
    Cargo:      {v} / {vmax} ({vleft} left)
    
    Value:     {val}
        """.format(
            t = self.time_left,
            system = self.system,
            station = self.station,
            c = c, cmax = cmax, cleft = cleft,
            v = v, vmax = vmax, vleft = vleft,
            val = self.value
        )
    
    def limits_left(self):
        total_vol = 0
        total_col = 0
        
        for item, (count, value) in self.cargo:
            total_vol += types[item]["volume"] * count
            total_col += value
        
        return (self.volume_limit - total_vol, self.collateral_limit - total_col)
    
    def validate(self):
        (v, c) = self.limits_left()
        assert v >= 0, 'Volume exceeded'
        assert c >= 0, 'Collateral exceeded'
        
        assert self.time_left >= 0, 'Out of time'
    
    @property
    def value(self):
        return self.wallet
        

# Helper class for incremental state modification
class MutableState:
    def __init__(self, other = None):
        if other is None:
            for name, prop in state_props.items():
                setattr(self, name, copy.copy(prop.default))
        else:
            for name, prop in state_props.items():
                setattr(self, name, prop.convert_out(getattr(other, name)))

In [114]:
# --- Marker broker simulation ---
def market_transaction(state, type_id, amount, order_type):
    assert order_type in ['buy', 'sell']
    
    # For the duration of this function, we need this element to exist
    if type_id not in state.cargo:
        state.cargo[type_id] = (0, 0)
    
    # Get all possible orders in appropriate order
    candidate_orders = orders[get_region_id(state.system)][type_id]
    candidate_orders = [o for o in candidate_orders if o.location_id == state.station]
    candidate_orders.sort(key = lambda x: x.price, reverse = order_type == 'sell')
    
    # Check that we only sell as many items as we have
    if order_type == 'sell':
        amount = min(amount, state.cargo[type_id][0])    
    
    factor = 1 if order_type == 'buy' else -1
    
    for order in candidate_orders:
        # Check if we even need anything more
        if amount == 0:
            break
        
        # Check if we can get something from this order
        remaining = order.amount_remaining
        
        if order.order_id in state.updated_orders:
            remaining = state.updated_orders[order.order_id]
                
        transfer = min(amount, remaining)
        
        # Don't buy more than we can afford or store on our ship
        if order_type == 'buy':
            transfer = min(
                transfer,
                int(state.wallet / order.price)
            )
    
            vol_left, coll_left = state.limits_left()
            can_store = int(min(
                vol_left / types[type_id].volume,
                coll_left / order.price
            ))
            
            transfer = min(
                transfer,
                can_store
            )
        
        if transfer == 0:
            continue
        
        # Conduct transaction
        delta = transfer if order_type == 'buy' else -transfer
        state.wallet -= delta * order.price
        
        (cnum, cval) = state.cargo[type_id]
        state.cargo[type_id] = (
            cnum + delta,
            cval + delta * order.price
        )
        
        # Record that the order was partially filled
        state.updated_orders[order.order_id] = remaining - transfer
        
        # Continue buying if neccessary
        amount -= transfer
    
    # Remove empty cargo categories
    if state.cargo[type_id][0] == 0:
        del state.cargo[type_id]

In [115]:
# Action calculus on states

warp_time = 1.0
accept_range = 5

max_volume = 1e6

# Wrapper that allows an action to just modify a mutable state representation
# and performs validity checking of resulting state
class ActionImpl:
    def __init__(self, f):
        self.wrapped = f
    
    def __call__(self, state):
        s = MutableState(state)
        s.last_modified_item = None
        self.wrapped(s)
        s = State(s)
        
        s.validate()
        
        return s
        
def action(f):
    return ActionImpl(f)

@functools.lru_cache(maxsize=None)
def warp_to(station_id):
    station = get_station(station_id)
    
    distances = nx.shortest_path_length(subgraph, station.system_id)
    
    @action
    def do_warp_to(s):
        assert s.station != station_id, 'Cannot warp from to same station'
        
        if s.system == station.system_id:
            t = warp_time
        else:
            #t = warp_time * nx.shortest_path_length(subgraph, s.system, station.system_id)
            t = warp_time * distances[s.system]
        
        s.station = station_id
        s.system = station.system_id
        
        s.time_left -= t
    
    return do_warp_to

@functools.lru_cache(maxsize=None)
def warp_to_system(system_id):
    distances = nx.shortest_path_length(subgraph, system_id)
    
    @action
    def do_warp_to_system(s):
        print(s)
        t = warp_time * system_distance(system_id, s.system)
        
        s.system = system_id
        s.station = None
        
        s.time_left -= t
    
    return do_warp_to_system
    

def buy(type_id, amount):
    @action
    def do_buy(s):
        market_transaction(s, type_id, amount, 'buy')
        
    return do_buy

def sell(type_id, amount):
    @action
    def do_sell(s):
        market_transaction(s, type_id, amount, 'sell')
    
    return do_sell

def actions(s):
    (vol, col) = s.limits_left()
    
    accept_actions = [
        accept_contract(c)
        for c in contracts.values()
        if c.volume <= vol and c.collateral <= col
    ]
    
    #jump_actions = [
    #    jump_to_system(sys)
    #    for sys in subgraph.neighbors(s.system)
    #]
    
    #dock_actions = [
    #    warp_to_station(sta)
    #    for sta in systems[s.system].get("stations", [])
    #]
    
    warp_targets = [
        contracts[cid].start_location_id
        for cid in s.accepted
    ] + [
        contracts[cid].end_location_id
        for cid in s.loaded
    ]
    
    warp_actions = [
        warp_to(t)
        for t in warp_targets
    ]
    
    load_actions = [
        load_contract(contracts[cid])
        for cid in s.accepted
        if s.station is not None and contracts[cid].start_location_id == s.station
    ]
    
    finish_actions = [
        finish_contract(contracts[cid])
        for cid in s.loaded
        if s.station is not None and contracts[cid].end_location_id == s.station
    ]
    
    result = warp_actions + load_actions + finish_actions + accept_actions
    
    return result

def apply_action(a, s):
    try:
        return a(s)
    except AssertionError as e:
        return None

def apply_actions(s):
    return [
        s2
        for s2 in [apply_action(a, s) for a in actions(s)]
        if s2 is not None
    ]    

In [116]:
# Build a node into a state space traversal graph by expanding every node that maximizes a given heuristic
def expand(root, heuristic, count, sort_every = 5):
    result = nx.DiGraph()
    result.add_node(root)
    
    unexpanded = [root]
    
    for counter in trange(0, count):
        if not unexpanded:
            break;
        s = unexpanded.pop()
        
        new_states = apply_actions(s)
        
        for s2 in new_states:
            if s2 not in result:
                result.add_node(s2)
                #unexpanded.append(s2)
                unexpanded.insert(0, s2)
            
            result.add_edge(s, s2)
        
        if counter % sort_every == 0:
            unexpanded.sort(key = heuristic)
        
        counter += 1

    return result, frozenset(unexpanded)

def get_values(root, g, unexpanded, heuristic):
    values = {}
    for node in nx.dfs_postorder_nodes(g, root):
        # Unexpanded nodes are weighted with their heuristic
        if node in unexpanded:
            values[node] = heuristic(node)
            continue
        
        # Expanded nodes are weighted with the maximum of successor nodes
        # and the known value of the node
        val = node.value()
        for n2 in g.successors(node):
            if n2 not in values:
                continue
                
            val = max(val, values[n2])
        
        values[node] = val
    
    return values

In [117]:
# Attention layers

def full_attention(keys, values, queries):
    """
        keys    - batch_shape + [n_sequence1, dim_k] tensor holding query keys
        values  - batch_shape + [n_sequence1, dim_v] tensor holding query results
        queries - batch_shape + [n_sequence2, dim_k] tensor holding queries

        Returns batch_shape + [n_sequence2, dim_v] tensor holding result
    """
    with tf.name_scope('scaled_dot_product_attention'):
        # Compute the [..., n_sequence2, n_sequence1] matrix of weights
        weights = tf.linalg.matmul(queries, keys, transpose_b = True)
        
        # Normalize against the key dimension
        dk = tf.cast(tf.shape(keys)[-1], tf.float32)
        weights = weights / tf.math.sqrt(dk)
        
        # Compute softmax over the sequence1 dimension
        weights = tf.nn.softmax(weights, axis = -1)
        
        # Apply weights to values
        result = tf.linalg.matmul(weights, values)
        
        return result


def reduced_attention(keys, values, queries):
    """
        keys    - batch_shape + [n_sequence1, dim_k] tensor holding query keys
        values  - batch_shape + [n_sequence1, dim_v] tensor holding query results
        queries - batch_shape + [n_sequence2, dim_k] tensor holding queries

        Returns batch_shape + [n_sequence2, dim_v] tensor holding result
    """
    with tf.name_scope('reduced_scaled_dot_product_attention'):
        # Compute the [..., dim_k, dim_v] matrix of weights
        weights = tf.linalg.matmul(keys, values, transpose_a = True)
        
        # Normalize against the key dimension
        dk = tf.cast(tf.shape(keys)[-1], tf.float32)
        weights = weights / tf.math.sqrt(dk)
        
        # Compute softmax over the dim_k dimension
        weights = tf.nn.softmax(weights, axis = -2)
        
        # Apply weights to values
        result = tf.linalg.matmul(queries, weights)
        
        return result

class MultiHeadedAttention(tf.keras.layers.Layer):
    def __init__(self, d_out, d_model, num_heads, f = reduced_attention):
        super(MultiHeadedAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
    
        self.wk = tf.keras.layers.Dense(num_heads * d_model, name = 'Wk')
        self.wv = tf.keras.layers.Dense(num_heads * d_model, name = 'Wv')
        self.wq = tf.keras.layers.Dense(num_heads * d_model, name = 'Wq')
        
        self.wout = tf.keras.layers.Dense(d_out, name = 'Wout')
        
        self.f = f
        
    def _split_heads(self, x):
        with tf.name_scope('split_heads'):
            # Ensure static shape
            #x.set_shape(tf.TensorShape([None, None, num_heads * d_model]))
            #static_shape = x.get_shape();

            # Dynamic reshape
            batch_size = tf.shape(x)[0]
            seq_len = tf.shape(x)[1]

            x = tf.reshape(x, [batch_size, seq_len, self.num_heads, self.d_model])

            # Static reshape
            # x.set_shape(static_shape[0:2] + tf.TensorShape([num_heads, d_model]))

            x = tf.transpose(x, [0, 2, 1, 3])
        return x
    
    def _merge_heads(self, x):
        with tf.name_scope('merge_heads'):
            batch_size = tf.shape(x)[0]
            seq_len    = tf.shape(x)[2]

            x = tf.transpose(x, [0, 2, 1, 3])
            x = tf.reshape(x, [batch_size, seq_len, self.num_heads * self.d_model])
            return x
        
    def call(self, k, v = None, q = None):
        if v == None:
            v = k
        
        if q == None:
            q = k

        k = self.wk(k)
        v = self.wv(v)
        q = self.wq(q)
        
        k = self._split_heads(k)
        v = self._split_heads(v)
        q = self._split_heads(q)
        
        result = self.f(k, v, q)
        result = self._merge_heads(result)
        
        return self.wout(result)

# --- Table join computation ---

# Joins orders with data for the associated systems and types
def orders_join(orders, systems, types):
    # Foreign keys
    order_types = tf.cast(orders[...,0], tf.int32)
    order_systems = tf.cast(orders[...,1], tf.int32)
    
    return tf.concat([
        orders,
        tf.gather(types, order_types),
        tf.gather(systems, order_systems)
    ], axis = -1)
    
# ------------------ Model class -----------------------------------------

class Model(tf.keras.Model):
    def __init__(self, n_layers, output_width = 128, n_heads = 16, head_dimension = 8):
        super(Model, self).__init__()
        self.ratt_layers = [
            MultiHeadedAttention(output_width, n_heads, head_dimension)
            for i in range(0, n_layers)
        ]
        
        self.type_data = tf.Variable(np.zeros([len(types), 10], dtype = np.float32))
        self.system_data = tf.Variable(np.zeros([len(subgraph), 10]), dtype = np.float32)
        
    # Args:
    #  orders: Tuple of 2 [None, None] ragged tensors describing types and systems of orders
    #          and one [None, None, n_order_data] ragged tensor giving static data for the market orders
    def call(self, orders):
        def apply_nobatch(layer, x):
            x = tf.expand_dims(x, axis = 0)
            x = layer(x)
            x = tf.squeeze(x, axis = 0)
            
            return x
        
        v = orders_join(orders, self.system_data, self.type_data)
        
        # Apply layers to input
        for layer in self.ratt_layers:
            with tf.name_scope('apply_nobatch'):
                v = tf.ragged.map_flat_values(
                    lambda x: apply_nobatch(layer, x),
                    v
                )
            
        return v     

In [72]:
# Solar system encoding
@functools.lru_cache(maxsize=None)
def encode_system(system_id, state = None):
    assert system_id in system_ids
    
    index = system_ids.index(system_id)
    system = systems[system_id]
    
    return [index, system["security_status"]] + [system_distance(l, system_id) for l in landmarks]

@functools.lru_cache(maxsize=None)
def encode_type(type_id, state = None):
    assert type_id in type_ids
    
    index = type_ids.index(type_id)
    t = types[type_id]
    
    return [
        index,
        t["volume"]
    ]

def can_encode_order(order):
    return order.type_id in type_ids and order.system_id in system_ids

# Encodes a [region, type] dict of orders
def encode_orders(orders):
    def encodeable(region_id, type_id):
        os = orders[region_id].get(type_id, [])
        
        return [o for o in os if can_encode_order(o)]
    
    print('Flattening data')
    flat_orders = [
        o
        for region_id in tqdm(region_ids)
        for type_id in types
        for o in encodeable(region_id, type_id)
        if o is not None
    ]
    
    row_lengths = [
        len(encodeable(region_id, type_id))
        for region_id in tqdm(region_ids)
        for type_id in types
    ]
        
    print('Encoding orders & transferring to GPU')
    flat_data = tf.constant([encode_order(o) for o in tqdm(flat_orders)], dtype = tf.float32)
    
    result = tf.RaggedTensor.from_uniform_row_length(
        tf.RaggedTensor.from_uniform_row_length(
            tf.RaggedTensor.from_row_lengths(
                flat_data,
                row_lengths
            ),
            len(type_ids)
        ),
        len(region_ids)
    )
    result = result[0]
    
    # Compute the delta base for later updates
    id_map = {
        order.order_id : idx
        for idx, order in enumerate(flat_orders)
    }
    
    delta_base = (id_map, flat_orders)
    
    return result, delta_base

# Encoding of a single order
def encode_order(order, state = None):
    tdata = encode_type(order.type_id, state)
    sdata = encode_system(order.system_id, state)
    
    if tdata is None or sdata is None:
        return None
    
    remaining = order.volume_remain
    
    
    if state is not None:
        updates = {k:v for k,v in state.updated_orders}
        
        if order.order_id in updates:
            remaining = updates[order.order_id]
    
    return [tdata[0], sdata[0]] + tdata[1:] + sdata[1:] + [
        order.location_id,
        remaining,
        1 if order.is_buy_order else 0,
        0 if order.is_buy_order else 1
    ]


def encode_delta(delta_base, state):
    if not state.updated_orders:
        return lambda x: x
    
    (id_map, flat_orders) = delta_base
    
    indices = [
        id_map[oid] for oid, _ in state.updated_orders
    ]
    
    updates = [
        encode_order(flat_orders[idx], state = state) for idx in indices
    ]
    
    indices = tf.constant([[i] for i in indices], dtype = tf.int32)
    updates = tf.constant(updates, dtype = tf.float32)
    
    def apply(x):        
        return tf.ragged.map_flat_values(
            tf.tensor_scatter_nd_update,
            x, indices, updates
        )
    
    return apply
        

encoded, delta_base = encode_orders(orders)

Flattening data



Encoding orders & transferring to GPU


In [118]:
#def pick_actions(
#    states, # [n_batch] array of states
#    action_params, # [n_batch, 3] tensor holding the logits for "buy", "sell", "move" respectively
#    move_params, # [n_batch, len(system_ids)] tensor holding the logits for systems to jump to for
#    buy_params,   # [n_batch, len(type_ids), 3] tensor holding in [...,0] the logits per type to buy, and in [...,1] and [...,2] mean and sdev. respectively of a Gaussian distribution determining the amount of items to be bought
#    sell_params,  # [n_batch, len(type_ids), 3] tensor for potential sales, analogously to buy_params (potential optimization: types ordered by state.cargo and not type_ids)
#):
#    # Shape enforcement
#    action_params.set_shape([len(states), 3])
#    move_params.set_shape([len(states), len(system_ids)])
#    buy_params.set_shape([len(states), len(type_ids), 3])
#    sell_params.set_shape([len(states), len(type_ids), 3])
#    
#    # Sample the distributions (TODO: Sampling all distributions, including the unneccessary ones, might prove slow, but not sure)
#    action_id = tfp.edward2.Categorical(logits = action_params)
#    move_system   = tfp.edward2.Categorical(logits = move_params)
#    
#    buy_type  = tfp.edward2.Categorical(logits = buy_params[...,0])
#    buy_amount = tfp.edward2.Normal(loc = buy_params[...,1], scale = buy_params[...,2])
#    
#    sell_type = tfp.edward2.Categorical(logits = sell_params[...,0])
#    sell_amount = tfp.edward2.Normal(loc = sell_params[...,1], scale = sell_params[...,2])
#    
#    # Create the actions
#    def pick_action(i):
#        choice = action_id.numpy()[i]
#        
#        if choice == 0:
#            # Jump action
#            idx = move_system.numpy()[i]
#            return warp_to_system(system_ids[idx])
#        elif choice == 1:
#            # Buy action
#            idx = buy_type[i].numpy()
#            amount = buy_amount[idx]
#            
#            return buy(type_ids[idx], amount.numpy())
#        elif choice == 2:
#            # Buy action
#            idx = sell_type[i].numpy()
#            amount = sell_amount[idx]
#            
#            return sell(type_ids[idx].numpy(), amount.numpy())
#    
#    actions = [pick_action(i) for i in range(0, len(states))]
#    
#    # Compute log probability of transition actions
#    def log_p(x):
#        return x.distribution.log_prob(x.value)
#    
#    log_p_id   = log_p(action_id)
#    log_p_warp = log_p(move_system)
#    log_p_buy  = log_p(buy_type) + log_p(buy_amount[:,buy_type.value])
#    log_p_sell = log_p(sell_type) + log_p(sell_amount[:,sell_type.value])
#    
#    log_p_tot = log_p_id
#    log_p_tot += tf.where(action_id == 0, log_p_warp, 0)
#    log_p_tot += tf.where(action_id == 1, log_p_buy, 0)
#    log_p_tot += tf.where(action_id == 2, log_p_sell, 0)
#    
#    return actions, log_p_tot

def sample_from(state, action_params, move_params, buy_params, sell_params):
    Joint = tfp.distributions.JointDistributionCoroutine
    
    def model():
        action_id = yield Joint.Root(tfp.distributions.Categorical(logits = action_params))
        
        if action_id == 0:
            target = yield tfp.distributions.Categorical(logits = move_params)
            target = system_ids[target.numpy()]
            
            return warp_to_system(target)
        
        if action_id == 1:
            params = buy_params
        elif action_id == 2:
            params = sell_params
            
        buy_idx = yield tfp.distributions.Categorical(logits = params[...,0])
        buy_amount = yield tfp.distributions.Normal(loc = params[...,1], scale = params[...,2])
        
        if action_id == 1:
            return buy(type_ids[idx].numpy(), amount.numpy())
        elif action_dd == 2:
            return sell([type_id for type_id in state.cargo][idx.numpy()], amount.numpy())
        
        assert False
    
    dist = Joint(model)
    
    sample = dist.sample()
    log_p = dist.log_prob(sample)
        
    # Interpret sample
    generator = model()
    try:
        next(generator)
        for s in sample:
            generator.send(s)
    except StopIteration as s:
        return s.value, log_p

#def sample_chain(states, length, model, history):
#    log_p_tot = tf.constant(np.zeros([len(starts)], dtype = tf.float32))
#    
#    for i in range(0, length):
#        # Encode states
#        state_data = encode_states(states)
#        
#        control, history = model(state_data, history)
#        
#        actions, log_p = pick_actions(starts, *control)
#        
#        states = [action(state) for action, state in zip(actions, states)]
#        log_p_tot += log_p
#    
#    return states, log_p_tot

test_state = MutableState()
test_state.cargo = {
    34: (10, 0) # 10 units of Tritanium
}
test_state.volume_limit = 8900
test_state.time_left = 100

test_state.system = root_id # in jita

state = State(test_state)
action_params = tf.constant([0, -1000, -1000], dtype = tf.float32)
system_logits = tf.constant([
        0 if sid in landmarks else -1000
        for sid in system_ids
    ],
    dtype = tf.float32
)
buy_params = tf.zeros([len(type_ids), 3], dtype = tf.float32)
sell_params = tf.zeros([len(type_ids), 3], dtype = tf.float32)

print(state)

for i in trange(0, 1000):
    chosen_action, log_p = sample_from(state, action_params, system_logits, buy_params, sell_params)
    print(state)
    state = chosen_action(state)


State:
    Time left: 100
    
    System:  30000142
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        



State:
    Time left: 100
    
    System:  30000142
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        

State:
    Time left: 73.0
    
    System:  30002510
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        

State:
    Time left: 55.0
    
    System:  30004969
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        

State:
    Time left: 43.0
    
    System:  30000142
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        

State:
    Time left: 16.0
    
    System:  30002510
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        



AssertionError: Out of time

In [130]:
test_state = MutableState()

idgen = iter(delta_base[0])
idgen = itertools.islice(idgen, 300)

for i in idgen:
    test_state.updated_orders[i] = 10
    
#delta = encode_delta(delta_base, State(test_state))

#for i in trange(0, 1000):
#    delta = encode_delta(delta_base, State(test_state))
#    encoded_2 = delta(encoded)

encoded_2 = [
    encode_delta(delta_base, State(test_state)) for i in trange(0, 1000)
]

#diff = encoded_2 - encoded

#diff.flat_values[0:3]

In [132]:
encoded_3 = [delta(encoded) for delta in tqdm(encoded_2)]

In [9]:
rval = tfp.edward2.Categorical(logits = [0, 0, 0])

d:\programme\python 3.6.6\lib\site-packages\ipykernel_launcher.py:1: UserWarning: tfp.edward2 module is deprecated and will be removed on 2019-12-01. Use https://github.com/google/edward2 library instead.
  """Entry point for launching an IPython kernel.


In [57]:
m = Model(3)

@tf.function
def f(x):
    y = m(x)
    return y

import datetime
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func'
writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = f(encoded_orders)

with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace_2",
        step=0,
        profiler_outdir=logdir)


In [13]:
tf.summary.trace_off()

In [26]:
for o in orders.values():
    print(o[0].keys())
    break

dict_keys(['duration', 'is_buy_order', 'issued', 'location_id', 'min_volume', 'order_id', 'price', 'range', 'system_id', 'type_id', 'volume_remain', 'volume_total'])


In [ ]:
r = tf.ragged.constant([[[0]], [[2], [3]]])

In [ ]:
r.to_tensor()

In [ ]:
s = MutableState()

s.system = root_id
s.time_left = 40.0

ship = 'tayra'

if ship == 'charon':
    s.volume_limit = 1350000
    s.collateral_limit = 750000000
elif ship == 'tayra':
    s.volume_limit = 8395
    s.collateral_limit = 100000000

expanded = set([])

def heur(x):
    return x.value()

g, unexp = expand(State(s), heur, 30000)

heurmax = max(*[heur(n) for n in g])
print(heurmax)

In [ ]:
import cProfile

cProfile.run("expand(State(s), heur, 10000)")

In [ ]:
cycles = nx.simple_cycles(g);

for c in cycles:
    print(c)

In [ ]:
plt.figure(figsize=(20, 20))

nx.draw_networkx(
    g,
    labels = {n:systems[n.system]["name"] for n in g},
    node_size = [300 * heur(n) / heurmax for n in g],
    node_color = [heur(n) / heurmax for n in g],
    cmap = 'jet'
)

c = matplotlib.colorbar.ColorbarBase(plt.gca(), cmap='jet')

In [ ]:
vals = get_values(State(s), g, unexp, heur)

In [ ]:
len(g)

In [ ]:
plt.figure(figsize=(20, 20))

valmax = max(*vals.values())

subg = g.subgraph([n for n in g if vals[n] == valmax])

nx.draw_networkx(
    subg,
    labels = {n:systems[n.system]["name"] for n in subg},
    node_size = [30 * n.time_left for n in subg],
    node_color = [300 * heur(n) / heurmax for n in subg],
    alpha = 1.0,
    cmap = 'jet'
)

plt.show()

for n in nx.dfs_preorder_nodes(g, State(s)):
    if vals[n] < valmax:
        continue
    
    print(n)

In [ ]:
for n in nx.dfs_preorder_nodes(g, State(s)):
    if not n.loaded:
        continue
    
    print(n)

In [ ]:
plt.hist([heur(n) for n in g]);

In [ ]:
vals

In [ ]:
systems[root_id]

In [20]:
import pyswagger

dir(pyswagger.primitives.Model)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_with',
 'cleanup',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [26]:
with open('orders.dat', 'wb') as f:
    pickle.dump(orders, f)

KeyError: '__getstate__'

In [23]:
type(orders)

dict

In [30]:
df

KeyError: '__next__'

KeyError: '__next__'